##Titanic: Uczenie maszynowe związane z katastrofą


titanic_train.csv



In [ ]:
#Importowanie biblioteki
import pandas as pd
#
import io
from google.colab import files
uploaded = files.upload()

#ścieżka do wczytania pliku
titanic_train = io.BytesIO(uploaded['titanic_train.csv'])

#Wczytanie zmiennej
train = pd.read_csv(titanic_train)

Saving titanic_train.csv to titanic_train.csv


In [ ]:
#Sprawdzenie pliku
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#Przygotowanie danych 


In [ ]:
#Zmienna opisująca przeżycie katastrofy
y = train['Survived']
print(y)

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


In [ ]:
#eliminicja danych
interesujace_nas_kolumny = ["Pclass", "Sex", "Age"]

X = train[interesujace_nas_kolumny]
#podstawowe dane z tabeli
print(X.describe())
print(X.head())

           Pclass         Age
count  891.000000  714.000000
mean     2.308642   29.699118
std      0.836071   14.526497
min      1.000000    0.420000
25%      2.000000   20.125000
50%      3.000000   28.000000
75%      3.000000   38.000000
max      3.000000   80.000000
   Pclass     Sex   Age
0       3    male  22.0
1       1  female  38.0
2       3  female  26.0
3       1  female  35.0
4       3    male  35.0


In [ ]:

#zmiana wartości w kolumnie "Age". Nie jesteśmy w stanie stwierdzić wieku u wszystkich osób ze względu na brak danych w związku z czym należy przyjąć wiek średni osób na statku wynoszący 29.699118
X['Age'].fillna(X['Age'].mean(),inplace=True)
#Modyfikacja kolumny "Sex". Zamieniam występowanie stringów "male" i "female" na kolejno "1" i "0"
X['Sex'] = X['Sex'].replace('male', 1)
X['Sex'] = X['Sex'].replace('female', 0)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4535: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

#Wybór modelu
Wybieram jedną z czterech opcji modeli uczenia maszynowego. Działanie ma na celu dążenie do minimalizacji błędu w kolejnych krokach.

In [ ]:
#Importowanie bibliotek odpowiedzialnych na różne modele uczenia maszynowego
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


#Tworzenie modeli
titanic_model_decision_tree = DecisionTreeRegressor(random_state = 1)
titanic_model_tree_classifier = DecisionTreeClassifier(random_state = 1)
titanic_model_svc = SVC(C = 50, kernel = 'rbf', random_state = 1)
titanic_model_neighbors = KNeighborsClassifier(5)

In [ ]:
#Importowanie train_test_split
from sklearn.model_selection import train_test_split

#Przypisanie nowych zmiennych dla funkcji train_test_split z wykorzystaniem danych "X" i "y"
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)

In [ ]:
#Dopasowanie modeli z wykorzystaniem danych treningowych
titanic_model_decision_tree.fit(train_X, train_y)
titanic_model_tree_classifier.fit(train_X, train_y)
titanic_model_svc.fit(train_X, train_y)
titanic_model_neighbors.fit(train_X, train_y)



KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
#Importowanie narzędzi do liczenia błędu bezwględnego
from sklearn.metrics import mean_absolute_error

#Przypisywanie do nowej zmiennej wartości błędu bezwględnego konkretnego modelu na podstawie walidacyjnych danych 'y' oraz prognozowanych danych konkretnego modelu
val_mea_decision_tree = mean_absolute_error(val_y, titanic_model_decision_tree.predict(val_X))
val_mea_tree_classifier = mean_absolute_error(val_y, titanic_model_tree_classifier.predict(val_X))
val_mea_svc = mean_absolute_error(val_y, titanic_model_svc.predict(val_X))
val_mea_neighbors = mean_absolute_error(val_y, titanic_model_neighbors.predict(val_X))

#Wyprintowanie wartości poszczególnych błędów
print("Wartości błędu bezwzględnego dla poszczególnych modeli:")
print("DecisionTreeRegressor: ",val_mea_decision_tree)
print("DecisionTreeClassifier:",val_mea_tree_classifier)
print("SVC: \t\t\t",val_mea_svc)
print("KNeighborsClassifier",val_mea_neighbors)

Wartości błędu bezwzględnego dla poszczególnych modeli:
DecisionTreeRegressor:  0.2631617497430873
DecisionTreeClassifier: 0.21973094170403587
SVC: 			 0.2242152466367713
KNeighborsClassifier 0.2556053811659193


W związku z powyższym fragmentem kodu, decyduje się na wykorzystanie DecisionTreeClassifier w celu testowania modelu.

##Wgrywanie pliku testowego

In [ ]:
#Poniższy fragment kodu jest analogiczny z wgrywaniem pliku "titanic_train.csv"
uploaded = files.upload()

#Ścieżka do wczytania pliku
titanic_test = io.BytesIO(uploaded['titanic_test.csv'])

#Wczytanie pliku do zmiennej train
test = pd.read_csv(titanic_test)

Saving titanic_test.csv to titanic_test.csv


In [ ]:
#Sprawdzam jak wygląda wczytany plik oraz jakie kolumny są w nim zawarte
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Należy zaznaczyć, że tym razem nie mamy informacji o tym kto przeżył katastrofę. Model będzie musiał wyznaczyć te informacje.

#Przygotowanie danych testowych
Działania analogiczne do pliku z danymi treningowymi

In [ ]:
X_test = test[interesujace_nas_kolumny]

X_test['Age'].fillna(X_test['Age'].mean(),inplace=True)
X_test['Age'].fillna(X_test['Age'].mean(),inplace=True)
#Modyfikacja kolumny "Sex". Zamieniam występowanie stringów "male" i "female" na kolejno "1" i "0"
X_test['Sex'] = X_test['Sex'].replace('male', 1)
X_test['Sex'] = X_test['Sex'].replace('female', 0)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4535: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [ ]:
print(X_test.describe())
print(X_test.head())

           Pclass         Sex         Age
count  418.000000  418.000000  418.000000
mean     2.265550    0.636364   30.272590
std      0.841838    0.481622   12.634534
min      1.000000    0.000000    0.170000
25%      1.000000    0.000000   23.000000
50%      3.000000    1.000000   30.272590
75%      3.000000    1.000000   35.750000
max      3.000000    1.000000   76.000000
   Pclass  Sex   Age
0       3    1  34.5
1       3    0  47.0
2       2    1  62.0
3       3    1  27.0
4       3    0  22.0


#Prognozowanie wyniku na podstawie danych testujących


In [ ]:
prognozuj_wynik = titanic_model_tree_classifier.predict(X_test)

In [ ]:
#Tworzenie DataFrame zawierającej numery pasażerów oraz prognozowany wynik modelu
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': prognozuj_wynik})

In [ ]:
#Konwersja DataFrame z wygenerowanymi danymi do pliku .csv
submission.to_csv('Wynik.csv', index = False)